In [175]:
import numpy as np

# Creating a class "tree", which recursively stores information about each node
class tree:

    def __init__(self):

        self.best_feature = 0
        self.best_threshold = 0
        self.value = 0
        self.left = None
        self.right = None

def get_best_split(X, y):
    """
    Returns the best split of the data-set, as well as information about the
    nature (i.e. which feature and threshold were found) of this split.
    Parameters
    ----------
    X : Array of float64 (M,N)
        FEATURE AND SAMPLE MATRIX.
    y : Array of float64 (N)
        LABELS CORRESPONDING TO SAMPLES IN X.
    Returns
    -------
    best_X_left : Array of float64 (M,N_L)
    best_y_left : Array of float64 (N_L)
        SMALLER BEST FEATURE SAMPLES THAN THE THRESHOLD OF THE SPLIT.
    best_X_right : Array of float64 (M,N-N_L)
    best_y_right : Array of float64 (N-N_L)
        LARGER BEST FEATURE SAMPLES THAN THE THRESHOLD OF THE SPLIT.
    best_feature : int
        FEATURE INDEX WHICH DELIVERED THE SMALLEST LOSS FOR A SPLIT.
    best_threshold : float
        THRESHOLD VALUE WHICH DELIVERED THE SMALLEST LOSS FOR A SPLIT.
    """

    mse = np.inf

    # Checking all possible splits for their mean squared error
    for i in range(len(X[0,:])):

        # Unique and sorted list of the samples in X for feature i
        X_sampletypes_sorted = list(set(X[X[:,i].argsort()][:,i]))

        for j in range(len(X_sampletypes_sorted)-1):

            # Calculating the current split threshold
            split_threshold = (X_sampletypes_sorted[j] + X_sampletypes_sorted[j+1]) / 2

            # Splitting the data along the threshold value
            X_left = X[X[:,i] <= split_threshold]
            y_left = y[X[:,i] <= split_threshold]
            X_right = X[X[:,i] > split_threshold]
            y_right = y[X[:,i] > split_threshold]

            # Calculating the mse from mean under the current split
            mse_split = (np.sum(np.square(y_right - np.mean(y_right)))
                        + np.sum(np.square(y_left - np.mean(y_left))))/len(y)

            # Redefining current best split if mse is lower than previous best
            if (mse_split < mse):

                mse = mse_split

                best_X_right = X_right
                best_y_right = y_right
                best_X_left = X_left
                best_y_left = y_left

                best_threshold = split_threshold
                best_feature = i

    # Returning split data set with lowest mse
    return best_X_left, best_y_left, best_X_right, best_y_right, best_feature, best_threshold

def add_add_new_layer(X, y, max_depth, depth):
    """
    Recursively building the tree and storing information about each node.
    Parameters
    ----------
    X : Array of float64 (M,N)
        FEATURE AND SAMPLE MATRIX.
    y : Array of float64 (N)
        LABELS CORRESPONDING TO SAMPLES IN X.
    max_depth : int
        MAXIMAL DEPTH TO STOP ITERATION AT.
    depth : int
        CURRENT DEPTH OF THE TREE (I.E. AMOUNT OF LAYERS BUILT).
    Returns
    -------
    tree_build : Object
        FITTED TREE.
    """

    tree_build = tree()
    tree_build.value = np.mean(y)
    
    if depth < max_depth:

        # Obtaining the best split
        X_left, y_left, X_right, y_right, best_feature, best_threshold = get_best_split(X,y)

        # Recursively adding new layers to the tree
        if len(y_left) > 1 and len(y_right) > 1:

            tree_build.best_feature = best_feature
            tree_build.best_threshold = best_threshold
            tree_build.left = add_add_new_layer(X_left, y_left, max_depth, depth + 1)
            tree_build.right = add_add_new_layer(X_right, y_right, max_depth, depth + 1)

    return tree_build

def fit(X, y, max_depth):
    """
    Fit a tree to the given data set
    Parameters
    ----------
    X : Array of float64 (M,N)
        FEATURE AND SAMPLE MATRIX.
    y : Array of float64 (N)
        LABELS CORRESPONDING TO SAMPLES IN X.
    max_depth : int
        MAXIMAL DEPTH TO STOP ITERATION AT.
    Returns
    -------
    fitted_tree : Object
        FITTED TREE.
    """

    fitted_tree = add_add_new_layer(X, y, max_depth, depth=0)

    return fitted_tree

def predict(X, fitted_tree):
    """
    Fit a tree to the given data set
    Parameters
    ----------
    X : Array of float64 (M,N)
        FEATURE AND SAMPLE MATRIX.
    fitted_tree : Object
        TREE TO BE USED TO MAKE THE PREDICTIONS (MUST BE FITTED IN ADVANCE).
    Returns
    -------
    y_pred : Array of float64 (N)
        PREDICTIONS FOR X.
    """

    N = len(X[:,0])
    y_pred = np.empty(N)

    for i in range(N):

        temp_tree = fitted_tree

        # Following the branches according the properties of X's samples
        while temp_tree.left:

            # Left branch, if the current feature is smaller than threshold
            if X[i,temp_tree.best_feature] <= temp_tree.best_threshold:

                temp_tree = temp_tree.left

            # Otherwise, right branch
            else:

                temp_tree = temp_tree.right

        y_pred[i] = temp_tree.value

    return y_pred

In [176]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import time
%load_ext autoreload
%autoreload 2
from proj1_helpers import *
from implementations import *

# Loading the training data
y, tX, ids = load_csv_data('data/train.csv')
tX = standardize(tX)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [181]:
import numpy as np

class nodes:

    def __init__(self):

        self.left = None
        self.right = None
        self.feature = None
        self.threshold = None
        self.loss = None
        self.layer = None
        self.prediction = None

class DecisionTreeClassifier:

    def __init__(self, max_layers=10, min_samples_leaf=20):

        self.min_samples_leaf = min_samples_leaf
        self.max_layers = max_layers
        self.fitted_tree = None

    @staticmethod
    def _gini(y, N):

        N_1 = sum(y == 1)

        return 1 - (N_1/N)**2 - ((N - N_1)/N)**2

    def _gini_of_split(self, y, N, index):

        gini_left = self._gini(y[:index], index + 1)
        gini_right = self._gini(y[index:], N - index - 1)

        return ((index + 1)*gini_left + (N - index - 1)*gini_right) / N

    def _best_split(self, y, tX):

        best_gini = 1
        N = y.shape[0]
        indices = np.arange(self.min_samples_leaf, N - self.min_samples_leaf)

        for feature in range(tX.shape[1]):

            sorted_indices = np.argsort(tX[:, feature])
            y_sorted = y[sorted_indices]

            for index in indices[::100]:

                gini_split = self._gini_of_split(y_sorted, N, index)

                if gini_split < best_gini:

                    best_gini = gini_split
                    best_feature = feature
                    best_index = index
                    best_sorted_indices = sorted_indices

        best_threshold = tX[best_index, best_feature]
        indices_left = best_sorted_indices[:best_index]
        indices_right = best_sorted_indices[best_index:]

        return best_feature, best_threshold, best_gini, indices_left, indices_right

    def _add_new_layer(self, y, tX, layer=0):

        node = nodes()
        node.feature, node.threshold, node.loss, left, right = self._best_split(y, tX)
        node.layer = layer

        print("----------------")
        print("Layer ", layer)
        print("y = -1: ", sum(y == -1))
        print("y = 1: ", sum(y == 1))

        if ((node.layer < self.max_layers) and
            (left.shape[0] > 2*self.min_samples_leaf) and
            (right.shape[0] > 2*self.min_samples_leaf)):

            node.left = self._add_new_layer(y[left], tX[left], node.layer + 1)
            node.right = self._add_new_layer(y[right], tX[right], node.layer + 1)

        else:

            node.prediction = 1 if (2*sum(y == 1) > y.shape[0]) else -1

        return node

    def _goto_next_layer(self, node, sample):

        if node.prediction is not None:

            return node.prediction

        elif sample[self.fitted_tree.feature] < self.fitted_tree.threshold:

            return self._goto_next_layer(node.left, sample)

        else:

            return self._goto_next_layer(node.right, sample)

    def fit(self, y, tX):

        self.fitted_tree = self._add_new_layer(y, tX)

    def predict(self, tX):

        y_pred = np.empty(tX.shape[0])

        for index in range(tX.shape[0]):

            y_pred[index] = self._goto_next_layer(self.fitted_tree, tX[index])

        return y_pred

In [182]:
reg = DecisionTreeClassifier(max_layers = 5, min_samples_leaf = 20)

reg.fit(y[:10000], tX[:10000, :])

y_pred = reg.predict(tX[10000:20000, :])

----------------
Layer  0
y = -1:  6628
y = 1:  3372
----------------
Layer  1
y = -1:  3571
y = 1:  449
----------------
Layer  2
y = -1:  2875
y = 1:  245
----------------
Layer  3
y = -1:  2099
y = 1:  121
----------------
Layer  4
y = -1:  1935
y = 1:  85
----------------
Layer  5
y = -1:  1849
y = 1:  71
----------------
Layer  5
y = -1:  86
y = 1:  14
----------------
Layer  4
y = -1:  164
y = 1:  36
----------------
Layer  5
y = -1:  87
y = 1:  33
----------------
Layer  5
y = -1:  77
y = 1:  3
----------------
Layer  3
y = -1:  776
y = 1:  124
----------------
Layer  4
y = -1:  241
y = 1:  79
----------------
Layer  5
y = -1:  179
y = 1:  41
----------------
Layer  5
y = -1:  62
y = 1:  38
----------------
Layer  4
y = -1:  535
y = 1:  45
----------------
Layer  2
y = -1:  696
y = 1:  204
----------------
Layer  3
y = -1:  286
y = 1:  134
----------------
Layer  4
y = -1:  127
y = 1:  93
----------------
Layer  5
y = -1:  57
y = 1:  63
----------------
Layer  5
y = -1:  70
y = 

In [184]:
print(sum(y_pred == y[10000:20000]) / y_pred.shape[0])
print(sum(y_pred == -1), sum(y_pred == 1))
print(sum(y[10000:20000] == -1), sum(y[10000:20000] == 1))

0.6549
10000 0
6549 3451


In [173]:
from sklearn.datasets import make_classification as mc

reg = DecisionTreeClassifier(max_layers = 8, min_samples_leaf = 10)

tX, y = mc(n_samples = 1000, n_features=5)
y[y == 0] = -1

reg.fit(y[:700], tX[:700])

y_pred = reg.predict(tX[700:])

print(sum(y_pred == y[700:]) / y_pred.shape[0])
print(sum(y_pred == -1), sum(y_pred == 1))
print(sum(y[700:] == -1), sum(y[700:] == 1))

----------------
Layer  0
y = -1:  338
y = 1:  362
----------------
Layer  1
y = -1:  314
y = 1:  51
----------------
Layer  1
y = -1:  24
y = 1:  311
----------------
Layer  2
y = -1:  13
y = 1:  23
----------------
Layer  2
y = -1:  11
y = 1:  288
----------------
Layer  3
y = -1:  7
y = 1:  56
----------------
Layer  4
y = -1:  4
y = 1:  19
----------------
Layer  4
y = -1:  3
y = 1:  37
----------------
Layer  3
y = -1:  4
y = 1:  232
----------------
Layer  4
y = -1:  3
y = 1:  53
----------------
Layer  5
y = -1:  3
y = 1:  21
----------------
Layer  5
y = -1:  0
y = 1:  32
----------------
Layer  4
y = -1:  1
y = 1:  179
0.87
132 168
159 141
